In [3]:
!pip install datasets peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.1/280.1 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This be

In [22]:
import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, pipeline, logging
from peft import LoraConfig
from trl import SFTTrainer

base_model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
amazon_products_dataset = "ckandemir/amazon-products"
new_model ="llama-1.1B-chat-amazon-products"

dataset = load_dataset(amazon_products_dataset, split="train")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
)

model.config.use_cache = False
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # pad sequences
tokenizer.padding_side = "right" # right pad sequences

In [23]:
# run inference
logging.set_verbosity(logging.CRITICAL)
prompt = "Describe the product Hot Wheels 2019 Advent Calendar Vehicles."
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)
result = pipe(f"{prompt}")
print(result[0]["generated_text"])
#

Describe the product Hot Wheels 2019 Advent Calendar Vehicles.
The 2019 Hot Wheels Advent Calendar is a 36-piece set of 1:64 scale die-cast cars. The set includes 12 cars, each with a different theme. The cars are all based on the 2019 Hot Wheels lineup, and include cars from the 2019 Monster Jam, Monster Jam


In [16]:
#LoRA
peft_params = LoraConfig(
    lora_alpha=16, # multiplier of Lora outpul when its added to the full forward output
    lora_dropout=0.1, # with a probability of 10$ it will set random Lora output to 0
    r=32, # rank of Lora so matrices will have either LHS or RHS dimension of 32
    bias="none", # no bias term
    task_type="CAUSAL_LM"
)

training_args = TrainingArguments(output_dir='./results',
    num_train_epochs=2, # One pass over the dataset
    per_device_train_batch_size=1, # mbs=1
    gradient_accumulation_steps=16, # effective batch size 16*1
    optim="adamw_torch",
    save_steps=100, # checkpoint every 25 steps
    logging_steps=100,
    learning_rate=3e-4, # step size in the optimizer update
    weight_decay=0.001,
    fp16=True, # 16 bit
    bf16=False, # not supported on V100
    max_grad_norm=0.5, # gradient clipping improves convergence
    max_steps=-1,
    warmup_ratio=0.05, # learning rate warmup
    group_by_length=True,
    lr_scheduler_type="cosine", #cosine lr scheduler
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params, # parameter efficient fine tuning AKA Lora
    dataset_text_field="Description",
    max_seq_length=256,
    tokenizer=tokenizer,
    args=training_args,
    packing=False
)

import gc # garbage collection
gc.collect()
torch.cuda.empty_cache() #clean cache

trainer.train() #train the model
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/23993 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


{'loss': 3.7529, 'grad_norm': 7.360247611999512, 'learning_rate': 0.000198, 'epoch': 0.06668611678406201}
{'loss': 3.1767, 'grad_norm': 4.70203161239624, 'learning_rate': 0.0002997809377912442, 'epoch': 0.13337223356812403}
{'loss': 3.0836, 'grad_norm': 4.187793731689453, 'learning_rate': 0.00029800547420441565, 'epoch': 0.20005835035218605}
{'loss': 3.0363, 'grad_norm': 3.9570157527923584, 'learning_rate': 0.0002944220550244107, 'epoch': 0.26674446713624805}
{'loss': 2.9891, 'grad_norm': 3.060197353363037, 'learning_rate': 0.00028908308716683027, 'epoch': 0.3334305839203101}
{'loss': 2.9528, 'grad_norm': 2.7537288665771484, 'learning_rate': 0.00028205346943096366, 'epoch': 0.4001167007043721}
{'loss': 2.9119, 'grad_norm': 2.87355899810791, 'learning_rate': 0.0002734186516195989, 'epoch': 0.4668028174884341}
{'loss': 2.8587, 'grad_norm': 2.5957322120666504, 'learning_rate': 0.00026328359583833186, 'epoch': 0.5334889342724961}
{'loss': 2.8554, 'grad_norm': 2.839942216873169, 'learning_r

('llama-1.1B-chat-amazon-products/tokenizer_config.json',
 'llama-1.1B-chat-amazon-products/special_tokens_map.json',
 'llama-1.1B-chat-amazon-products/tokenizer.model',
 'llama-1.1B-chat-amazon-products/added_tokens.json',
 'llama-1.1B-chat-amazon-products/tokenizer.json')

In [29]:
prompt = "Describe the product Hot Wheels 2019 Advent Calendar Vehicles."
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=115
)
result = pipe(f'{prompt}')
print(result[0]['generated_text'])

Describe the product Hot Wheels 2019 Advent Calendar Vehicles.
The 2019 Hot Wheels Advent Calendar is a 36-piece set of 1:64 scale die-cast cars. The set includes 12 cars, each with a different theme. The cars are all based on the 2019 Hot Wheels lineup, and include cars from the 2019 Monster Jam, Monster Jam World Series, and Monster Jam Triple Threat Series.

